# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
city_data = pd.read_csv('City_Data.csv')


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

In [8]:
# Plot Heatmap

locations = city_data[["Lat", "Lng"]]
heat = city_data['Humidity']
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=heat, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
idealloc_df = city_data.loc[(city_data['Humidity'] >= 40) & (city_data['Humidity'] <= 70)
                            & (city_data['Max Temp'] <= 70) & (city_data['Wind Speed'] <= 10)].sample(40)
idealloc_df =  idealloc_df.dropna(how = 'any')
del idealloc_df['Unnamed: 0']
idealloc_df.head(25)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
361,coracora,-15.03,-73.78,7.43,51.0,0.0,0.94,PE,1595898667
7,albany,42.60,-73.97,32.78,70.0,5.0,2.40,US,1595898393
355,longyearbyen,78.22,15.64,17.00,55.0,99.0,4.60,SJ,1595898665
462,tigil,57.80,158.67,20.06,54.0,0.0,4.93,RU,1595898695
392,pangnirtung,66.15,-65.71,15.22,48.0,100.0,3.68,CA,1595898675
234,san juan,-31.54,-68.54,9.35,49.0,6.0,3.62,AR,1595898403
179,kaitangata,-46.28,169.85,8.89,70.0,100.0,3.58,NZ,1595898618
222,bilma,18.69,12.92,31.23,47.0,5.0,5.89,NE,1595898630
344,san rafael,-34.62,-68.33,7.15,57.0,0.0,2.01,AR,1595898663
412,jaen,38.00,-3.50,23.82,41.0,0.0,2.45,ES,1595898681


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = pd.DataFrame()
hotel_df['Hotel Name'] = 'NaN'

In [16]:
for i,locs in idealloc_df[['Lat','Lng','City','Country']].iterrows():
    target_coordinates = f'{locs[0]}, {locs[1]}'
    target_search = "Hotel"
    target_radius = 5000
#     target_type = "restaurant"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
#         "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # run a request using our params dictionary
    try:
        response = requests.get(base_url, params=params).json()
        print(f'Potential hotel {response["results"][0]["name"]}')
#         print(response['results'][0]['geometry']['location'])
        lat_lng = response['results'][0]['geometry']['location']
        hotel_df = hotel_df.append({'Hotel Name':response["results"][0]["name"],'City':locs[2],
                         'Country':locs[3],'Lat': lat_lng['lat'],'Lng': lat_lng['lng']}, ignore_index=True)
    except:
        print('No nearby locations') 
    

Potential hotel B&B La Perla Nera
No nearby locations
Potential hotel The Ritz-Carlton, Hong Kong
Potential hotel Grand Naniloa Hotel Hilo - a DoubleTree by Hilton
Potential hotel Auyuittuq Lodge (Inns North)
Potential hotel Zyryanka
Potential hotel Canopy by Hilton Minneapolis Mill District
No nearby locations
Potential hotel Otel' Yevropeyskiy
No nearby locations
Potential hotel Courtyard by Marriott Danbury
Potential hotel Private Villa Yui Terrace
Potential hotel Hostal Mi Refugio
Potential hotel Mantra Mackay
Potential hotel Hotel "Nord House"
Potential hotel Hotel Carpa Manzano
Potential hotel Sberbank Rossii
Potential hotel Havøysund Hotel & Rorbuer
Potential hotel Residence Inn by Marriott Waynesboro
Potential hotel Seeadler Bay Hotel


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Display figure
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# print(response['results'][0]['name'])
hotel_df